In [ ]:

from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import tensorflow as tf
import numpy as np 
import matplotlib 
import os
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, BatchNormalization , ReLU, LeakyReLU, Dropout,Reshape
from keras.optimizers import Adam
import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def discriminator():
	model = Sequential()
	model.add(Conv2D(8, (3,3) , strides= (2,2), padding= 'same',input_shape = (256,256,3)))
	model.add(LeakyReLU(0.2))
	model.add(Dropout(0.3))

#128*128*8
	
	model.add(Conv2D(16, (3,3) , strides= (2,2), padding= 'same'))
	model.add(BatchNormalization( momentum= 0.7))
	model.add(LeakyReLU(0.2))
	#model.add(Dropout(0.3))
#64*64*16

	model.add(Conv2D(32, (3,3) , strides= (2,2), padding= 'same'))
	#model.add(BatchNormalization(momentum= 0.7))
	model.add(LeakyReLU(0.2))
	#model.add(Dropout(0.3))
#32*32*32

	model.add(Conv2D(64, (3,3) , strides= (2,2), padding= 'same'))
	model.add(BatchNormalization(momentum= 0.7))
	model.add(LeakyReLU(0.2))
	model.add(Dropout(0.3))
#16*16*64

	model.add(Conv2D(128, (3,3) , strides= (2,2), padding= 'same'))
	#model.add(BatchNormalization(momentum= 0.7))
	model.add(LeakyReLU(0.2))
	#model.add(Dropout(0.3))
#8*8*128
	model.add(Conv2D(256, (3,3) , strides= (2,2), padding= 'same'))
	model.add(BatchNormalization(momentum= 0.7))
	model.add(LeakyReLU(0.2))
	model.add(Dropout(0.3))
#4*4*256
	model.add(Flatten())
	model.add(Dense(128))
	model.add(LeakyReLU(0.2))


	model.add(Dense(1,activation ='sigmoid'))
	optimus = Adam(learning_rate= 0.0004 ,beta_1=0.5)
	model.compile(optimizer= optimus, loss = 'binary_crossentropy',metrics=['accuracy'])

	return model

In [ ]:
def generator(latent_dim):
  
	model = Sequential()
	model.add(Dense((4096),input_dim = latent_dim))
	model.add(Reshape((1,1,4096)))

	model.add(Conv2DTranspose(512, (4,4) , strides=(2,2), padding='same'))
	model.add(ReLU())
#2*2*512
	model.add(Conv2DTranspose(256,(4,4),strides= 2, padding = 'same'))
	#model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))
#4*4*256
	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))
#8*8*128
	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	#model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))
#16*16*128
	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))
#32*32*128
	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	#model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))
#64*64*128	
	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	model.add(Dropout(0.3))
#128*128*128

	model.add(Conv2DTranspose(128,(4,4),strides= 2, padding = 'same'))
	model.add(BatchNormalization(momentum=0.7))
	model.add(ReLU())
	#model.add(Dropout(0.3))

#256*256*4
	model.add(Conv2D(3,(3,3),activation= 'tanh',padding = 'same'))
	optimus = Adam(learning_rate=0.0001,beta_1=0.5)

	model.compile(optimizer=optimus , loss = 'binary_crossentropy')

	return model

In [ ]:
def gan(g_model,d_model):

	d_model.trainable = False
	model =Sequential()
	model.add(g_model)
	model.add(d_model)
	optimus = Adam(learning_rate=0.0002,beta_1=0.5)
	model.compile(optimizer=optimus,loss='binary_crossentropy')
	return model

In [ ]:
def load_real_data(xsam):
    
    x= np.array(xsam).reshape(-1,256,256,3)
    x = (x-127.5)/127.5
    print(x.shape)
    return x

In [ ]:
def generate_real_samples(dataset , n_samples):

	for i in range (n_samples):
		index = np.random.randint(0 , dataset.shape[0] , n_samples)

		image = dataset[index]
		y = np.ones((n_samples,1))

		return image,y

In [ ]:
def generate_latent_sample(laten_dim,n_samples):
	
	x = np.random.randn(laten_dim*n_samples)
	x = x.reshape(n_samples,laten_dim)

	return x

In [ ]:


def generate_fake_samples(g_model, latent_dim,n_samples):
	
	x = generate_latent_sample(latent_dim,n_samples)
	x = g_model.predict(x)
	y = np.zeros((n_samples,1))

	return x,y

In [ ]:
def save_plot(examples ,epoch , n = 3):
	examples = (examples+1) / 2.0

	for i in range(n*n):
		pyplot.subplot(3,3,1+i)
		pyplot.axis('off')
		pyplot.imshow(examples[i])
	filename = 'generated_plot_e%0.3d.png' % (epoch+1)
	pyplot.savefig('/content/drive/My Drive/Project_GAN/Restart/' + filename)
	pyplot.close()

In [ ]:
def summarize_performance(epoch,g_model,d_model,dataset,latent_dim,n_samples = 132):

	x_real ,y_real = generate_real_samples(dataset,n_samples)
	_,acc_real = d_model.evaluate(x_real,y_real)

	x_fake,y_fake = generate_fake_samples(g_model,latent_dim,n_samples)
	_,acc_fake = d_model.evaluate(x_fake,y_fake)

	print(('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100)))
	save_plot(x_fake,epoch)
	#filename1 = 'generator_model.h5' 
	#filename2 = 'discriminator_model.h5'
	d_model.save('/content/drive/My Drive/Project_GAN/saved_models/discriminator_model.h5')

	g_model.save('/content/drive/My Drive/Project_GAN/saved_models/generator_model.h5')

In [ ]:
def train(g_model,d_model,gan_model, dataset ,latent_dim, n_epoch = 500, n_batch = 32):
	batch_per_epoch = int(dataset.shape[0]/n_batch)
	batch = int(n_batch/2)

	for i in range(n_epoch):
		for j in range (batch_per_epoch):

			x_real,y_real = generate_real_samples(dataset,batch)

			d_model_loss1,_ = d_model.train_on_batch(x_real,y_real)

			x_fake ,y_fake = generate_fake_samples(g_model,latent_dim,batch)

			d_model_loss2,_ =d_model.train_on_batch(x_fake,y_fake)

			x_gan = generate_latent_sample(latent_dim, n_batch)
			y_gan = np.ones((n_batch,1))

			g_model_loss = gan_model.train_on_batch(x_gan,y_gan)

			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f'% 
                  (i+1, j+1, batch_per_epoch, d_model_loss1, d_model_loss2, g_model_loss))
		if (i+1) % 10 ==0 :
			summarize_performance(i,g_model,d_model,dataset,latent_dim)

In [ ]:
xsam = []
training_data = []
IMG_SIZE = (256)

data_dir = "/content/drive/My Drive/Project_GAN/new_data"

In [ ]:
# for category in CATEGORIES:

path = os.path.join(data_dir) 

for img in tqdm(os.listdir(path)):
  img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_UNCHANGED)  # convert to array
  img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB )
  new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))  # resize to normalize data size
  training_data.append([new_array])  # add this to our training_data


100%|██████████| 4072/4072 [00:40<00:00, 100.89it/s]


In [ ]:
for features in training_data:
  
  xsam.append(features)
#  #   return xsam
print(np.array(xsam).shape)
print(np.array(training_data).shape)

(4072, 1, 256, 256, 3)
(4072, 1, 256, 256, 3)


In [ ]:
latent_dim = 4096
d_model = discriminator()
g_model = generator(latent_dim)
gan_model = gan(g_model,d_model)
d_model.summary()
g_model.summary()
print(np.array(training_data).shape)

dataset = load_real_data(xsam)
train(g_model,d_model,gan_model,dataset,latent_dim)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 8)       224       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)       

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
>106, 111/127, d1=0.454, d2=0.399 g=1.132
>106, 112/127, d1=0.530, d2=0.487 g=0.765
>106, 113/127, d1=0.539, d2=0.553 g=1.131
>106, 114/127, d1=0.349, d2=0.546 g=0.673
>106, 115/127, d1=0.532, d2=0.648 g=0.857
>106, 116/127, d1=0.358, d2=0.885 g=0.915
>106, 117/127, d1=0.585, d2=0.491 g=1.010
>106, 118/127, d1=0.525, d2=0.596 g=1.191
>106, 119/127, d1=0.564, d2=0.808 g=1.326
>106, 120/127, d1=0.833, d2=0.645 g=1.291
>106, 121/127, d1=0.557, d2=0.560 g=1.465
>106, 122/127, d1=0.696, d2=0.372 g=1.272
>106, 123/127, d1=0.728, d2=0.587 g=1.170
>106, 124/127, d1=0.436, d2=0.503 g=1.277
>106, 125/127, d1=0.796, d2=0.423 g=0.946
>106, 126/127, d1=0.390, d2=0.671 g=1.205
>106, 127/127, d1=0.790, d2=0.555 g=1.162
>107, 1/127, d1=0.917, d2=0.464 g=1.154
>107, 2/127, d1=0.844, d2=0.771 g=1.014
>107, 3/127, d1=0.348, d2=0.562 g=1.430
>107, 4/127, d1=0.593, d2=0.527 g=1.277
>107, 5/127, d1=0.568, d2=0.307 g=1.239
>107, 6/127, d1=0.243, d2=0.501 g=1

In [ ]:
from keras.models import load_model
latent_dim = 4096

dataset = load_real_data(xsam)
g_new_model = load_model('/content/drive/My Drive/Project_GAN/saved_models/generator_model.h5')
d_new_model = load_model('/content/drive/My Drive/Project_GAN/saved_models/discriminator_model.h5')
gan_model = gan(g_new_model,d_new_model)

train(g_new_model,d_new_model,gan_model,dataset,latent_dim)

(4072, 256, 256, 3)


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/127, d1=0.043, d2=0.091 g=2.690
>1, 2/127, d1=0.043, d2=0.168 g=3.025
>1, 3/127, d1=0.300, d2=0.006 g=2.477
>1, 4/127, d1=0.378, d2=0.266 g=3.485
>1, 5/127, d1=0.433, d2=0.322 g=2.574
>1, 6/127, d1=0.426, d2=0.262 g=3.053
>1, 7/127, d1=0.335, d2=0.265 g=3.617
>1, 8/127, d1=0.763, d2=0.156 g=3.025
>1, 9/127, d1=0.111, d2=0.203 g=2.219
>1, 10/127, d1=0.107, d2=0.412 g=2.991
>1, 11/127, d1=0.129, d2=0.128 g=1.913
>1, 12/127, d1=0.400, d2=0.073 g=1.991
>1, 13/127, d1=0.306, d2=0.796 g=2.364
>1, 14/127, d1=0.384, d2=0.443 g=3.481
>1, 15/127, d1=0.089, d2=0.137 g=2.750
>1, 16/127, d1=0.159, d2=0.037 g=2.086
>1, 17/127, d1=0.130, d2=0.147 g=2.357
>1, 18/127, d1=0.181, d2=0.229 g=2.166
>1, 19/127, d1=0.132, d2=0.276 g=2.158
>1, 20/127, d1=0.007, d2=0.037 g=2.973
>1, 21/127, d1=0.293, d2=0.209 g=2.275
>1, 22/127, d1=0.480, d2=0.085 g=2.244
>1, 23/127, d1=0.398, d2=0.562 g=3.632
>1, 24/127, d1=0.030, d2=0.192 g=2.982
>1, 25/127, d1=1.796, d2=0.026 g=2.093
>1, 26/127, d1=0.087, d2=0.545 g=2

KeyboardInterrupt: ignored